In [1]:
import torch
from torch import nn
from collections import OrderedDict

In [2]:
class CoDA_AE(nn.Module):
    
    def __init__(self, encoder_shape, decoder_shape):
        super(CoDA_AE, self).__init__()
        
        #define a list with each element the input and output dim of the layer
        layer_list = [[encoder_shape[i], encoder_shape[i+1]] for i in range(0, len(encoder_shape)-1)]
        
        encoder_dict = OrderedDict()

        for i in range(0,len(layer_list)):
            encoder_dict["layer"  + str(i)] = nn.ELU(nn.Linear(layer_list[i][0], layer_list[i][1]))

        self.encoder = nn.Sequential(encoder_dict)
        
        layer_list = [[decoder_shape[i], decoder_shape[i+1]] for i in range(0, len(decoder_shape)-1)]
        
        decoder_dict = OrderedDict()

        for i in range(0,len(layer_list)):
            decoder_dict["layer"  + str(i)] = nn.ELU(nn.Linear(layer_list[i][0], layer_list[i][1]))

        self.decoder = nn.Sequential(decoder_dict)
        
        
    def forward(x):
        #run the encoding and store the low level representation as A
        self.A = self.encoder(x)
        self.reconstruction = self.decoder(self.A)
        return self.reconstruction
        

def train(X, encoder_shape = [100,20], decoder_shape=[20, 100], epochs = 100):
    #ensure the first encoder dimension and final decoder dimension is equal to the size of the input
    assert X.shape(1) == encoder_shape[0] && X.shape(1) == decoder_shape[-1] 
    
    autoencoder = CoDA_AE(encoder_shape, decoder_shape)
    #define the CoDA-PCA loss
    loss_function = CoDA_Loss
    optim = torch.optim.SGD(lm.params, lr = 0.01)
    
    for epoch in range(epochs):
        
        out = model.forward(X)
        loss = loss_function(out, X)
        
        optim.zero_grad()
        
        loss.backward()
        optimiser.step()
        
        epoch += 1
        

        
CoDA_AE([100,200, 50, 40], [100,200])

CoDA_AE(
  (encoder): Sequential(
    (layer0): ELU(
      alpha=Linear(in_features=100, out_features=200, bias=True)
      (alpha): Linear(in_features=100, out_features=200, bias=True)
    )
    (layer1): ELU(
      alpha=Linear(in_features=200, out_features=50, bias=True)
      (alpha): Linear(in_features=200, out_features=50, bias=True)
    )
    (layer2): ELU(
      alpha=Linear(in_features=50, out_features=40, bias=True)
      (alpha): Linear(in_features=50, out_features=40, bias=True)
    )
  )
  (decoder): Sequential(
    (layer0): ELU(
      alpha=Linear(in_features=100, out_features=200, bias=True)
      (alpha): Linear(in_features=100, out_features=200, bias=True)
    )
  )
)

In [9]:
class CoDA_Loss(torch.nn.Module):
    
    def __init__(self):
        super(CoDA_Loss,self).__init__()
        
    def forward(self,Y,X):
        #X is original data, Y is CoDA reconstruction
        #input needs to be normalised by g(x) (geometric mean) for X_hat
        #TODO centering matrix?
        X_hat = X
        coda_loss = 0
        return coda_loss
